In [7]:
# Dependencies to Visualize the model
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
# Filepaths, numpy, and Tensorflow
import os
import numpy as np

In [12]:

import keras

Using TensorFlow backend.


ImportError: Traceback (most recent call last):
  File "C:\Users\bethf\Anaconda3\envs\xpressy\lib\site-packages\tensorflow\python\pywrap_tensorflow.py", line 58, in <module>
    from tensorflow.python.pywrap_tensorflow_internal import *
  File "C:\Users\bethf\Anaconda3\envs\xpressy\lib\site-packages\tensorflow\python\pywrap_tensorflow_internal.py", line 28, in <module>
    _pywrap_tensorflow_internal = swig_import_helper()
  File "C:\Users\bethf\Anaconda3\envs\xpressy\lib\site-packages\tensorflow\python\pywrap_tensorflow_internal.py", line 24, in swig_import_helper
    _mod = imp.load_module('_pywrap_tensorflow_internal', fp, pathname, description)
  File "C:\Users\bethf\Anaconda3\envs\xpressy\lib\imp.py", line 242, in load_module
    return load_dynamic(name, filename, file)
  File "C:\Users\bethf\Anaconda3\envs\xpressy\lib\imp.py", line 342, in load_dynamic
    return _load(spec)
ImportError: DLL load failed: The specified module could not be found.


Failed to load the native Tensor

In [ ]:
# Preprocessing imports
import glob
import os.path as path
import imageio
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

In [ ]:
# Model Imports
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.callbacks import EarlyStopping, TensorBoard
from sklearn.metrics import accuracy_score, f1_score

from datetime import datetime

In [ ]:
# root = 'samples'
root = 'Data'

# Load the images
file_paths = []
labels = []
targets = []

emotions = ['angry','disgust','fear','happy','neutral','sad','surprise']
x=0
e=0
for emotion in emotions:
    print(emotion)
    for filename in glob.glob(f"{root}/{emotion}/*.jpg"):
        path = filename.split('\\')[0]+'/'+filename.split("\\")[1]
        file_paths.append(path)
        labels.append(e)
        targets.append(emotion)
        x+=1
        print(x)
        if x == 1000:
            x = 0
            break
            
    for filename in glob.glob(f"{root}/{emotion}/*.png"):
        path = filename.split('\\')[0]+'/'+filename.split("\\")[1]
        file_paths.append(path)
        labels.append(e)
        targets.append(emotion)
        x+=1
        print(x)
        if x == 1000:
            x=0
            break
    e+=1
    x=0

In [ ]:
# Load the images
images = []
image_size = (50,50)

for path in file_paths:
    print(f"{x} of {len(file_paths)}")
    x+=1
    img = load_img(path,target_size = image_size,color_mode='grayscale')
    img = img_to_array(img)
    images.append(img)


In [ ]:
images = np.asarray(images)       
len(images)  

In [ ]:
images[0]

In [ ]:
# Get image size
image_size = np.asarray([images.shape[1], images.shape[2], images.shape[3]])
print(image_size)

In [ ]:
df = pd.DataFrame(list(zip(targets, labels,images,file_paths)),columns=["emotions","labels","images","paths"])
df.head()

In [ ]:
# Scale
images = images / 255


#Split into training and testing data
X = images
y = labels
from sklearn.model_selection import train_test_split
# 80/10/10 split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=1)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.111, random_state=1)

X_train.shape

In [ ]:
X_val.shape

In [ ]:
X_test.shape

In [ ]:
labels

In [ ]:
from keras.utils import to_categorical
num_classes=7

# Prepare y data
y_binary_train = to_categorical(y_train, num_classes=num_classes, dtype='float32')
y_binary_test = to_categorical(y_test, num_classes=num_classes, dtype='float32')

In [ ]:
y_binary_test

In [ ]:
# Define hyperparamters
MIN_NEURONS = 20
MAX_NEURONS = 120
KERNEL = (3, 3)
n_layers = 4
num_features = 64
# Training hyperparamters
EPOCHS = 150
BATCH_SIZE = 500
PATIENCE = 10
width, height = 50, 50
# Determine the # of neurons in each convolutional layer
steps = np.floor(MAX_NEURONS / (n_layers + 1))
nuerons = np.arange(MIN_NEURONS, MAX_NEURONS, steps)
nuerons = nuerons.astype(np.int32)

# Define a model
model = Sequential()

# Add convolutional layers
for i in range(0, n_layers):
    if i == 0:
        shape = (image_size[0], image_size[1], image_size[2])
        model.add(Conv2D(nuerons[i], KERNEL, input_shape=shape))
    else:
        model.add(Conv2D(nuerons[i], KERNEL))

    model.add(Activation('relu'))

# Add max pooling layer
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(MAX_NEURONS))
model.add(Activation('relu'))

# Add output layer
model.add(Dense(num_classes, activation='softmax'))

# Compile the model
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
# Print a summary of the model
model.summary()

In [ ]:
# Early stopping callback
early_stopping = EarlyStopping(monitor='loss', min_delta=0, patience=PATIENCE, verbose=2, mode='auto')

In [ ]:
# # TensorBoard callback
# LOG_DIRECTORY_ROOT = ''
# now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
# log_dir = "{}/run-{}/".format(LOG_DIRECTORY_ROOT, now)
# tensorboard = TensorBoard(log_dir=log_dir, write_graph=True, write_images=True)

In [ ]:
# Place the callbacks in a list
callbacks = [early_stopping]

In [ ]:
# Train the model
model.fit(np.array(X_train), np.array(y_train), epochs=EPOCHS, batch_size=BATCH_SIZE, callbacks=callbacks, verbose=3)

In [ ]:
# Make a prediction on the test set
test_predictions = model.predict(X_test)
test_predictions = np.argmax(test_predictions , axis=1)
test_predictions

In [ ]:
# Report the accuracy
accuracy = accuracy_score(y_test, test_predictions)
print("Accuracy: " + str(accuracy))

In [ ]:
y_binary_test

In [ ]:
 # Save the model
# model.save("emotion_model_trained.h5")